In [1]:
import torch
import torchvision.models as models
import monai

from torch.export import export, ExportedProgram
from torchvision.models.mobilenetv2 import MobileNet_V2_Weights
from executorch.backends.vulkan.partitioner.vulkan_partitioner import VulkanPartitioner
from executorch.exir import EdgeProgramManager, ExecutorchProgramManager, to_edge
from executorch.exir.backend.backend_api import to_backend
import executorch.backends.vulkan.serialization.vulkan_graph_schema as vk_graph_schema

/home/kindersc/Documents/model_conversion/executorch_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
densenet = getattr(monai.networks.nets, 'densenet121')
model = densenet(spatial_dims=2,
                    in_channels=3,
                    out_channels=3,
                    dropout_prob=float(0.2),
                    pretrained=True)

In [4]:
# Load the weights from the file
weights_path = '/home/kindersc/Documents/executorch_example/executorch/notebooks/best_metric_model.pth'
state_dict = torch.load(weights_path, map_location=torch.device('cpu'))

/tmp/ipykernel_85494/579817510.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path, map_location=torch.device('cpu'))


In [5]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [6]:
model.eval()

DenseNet121(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (layers): Sequential(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (dropout): Dropout2d(p=0.2, inplace=False)
        )
      )
      (denselayer2): _DenseLayer(
  

In [2]:

mobilenet_v2 = models.mobilenetv2.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT).eval()

In [7]:
sample_inputs = (torch.ones(1, 3, 256, 256),)

In [3]:
sample_inputs = (torch.randn(1, 3, 4, 4), )

In [4]:
compile_options = {"memory_layout_override": vk_graph_schema.VkMemoryLayout.TENSOR_CHANNELS_PACKED}
program: ExportedProgram = export(
    mobilenet_v2, sample_inputs,
)
edge_program: EdgeProgramManager = to_edge(program)
edge_program = edge_program.to_backend(VulkanPartitioner(compile_options))
executorch_program = edge_program.to_executorch()

INFO:root:Operators included in this Vulkan partition: 
INFO:root:    aten.sum.dim_IntList
INFO:root:    conv_with_clamp.default
INFO:root:    aten.div.Tensor
INFO:root:    aten.add.Tensor
INFO:root:    aten.full.default
INFO:root:    aten.linear.default
INFO:root:    aten.convolution.default
INFO:root:    aten.view_copy.default


In [5]:
with open("mobilenet_vulkan_tiny.pte", "wb") as file:
    executorch_program.write_to_file(file)

# Check outputs to verify they are same

In [6]:
mobilenet_v2(sample_inputs[0])

tensor([[-3.3535e-01, -9.6521e-02, -1.7967e-01,  3.6226e-01, -1.3325e-03,
         -1.7334e-01, -4.8555e-02, -3.6646e-01, -2.9623e-01,  4.9331e-02,
         -2.4724e-01, -9.1909e-02, -2.5554e-01, -1.8694e-01, -1.8966e-01,
         -3.7364e-01, -3.3307e-01, -1.5567e-01,  1.6443e-02, -2.0009e-01,
          6.0742e-03, -3.8055e-02,  3.7536e-01, -2.1140e-01, -1.3238e-01,
          2.9324e-02, -3.2338e-01, -7.2379e-01, -4.4837e-02, -9.5640e-02,
         -4.1326e-01, -4.6321e-01, -6.1448e-01, -4.3339e-01, -8.0298e-01,
         -3.6896e-01, -6.5389e-01, -4.6678e-02, -1.6993e-01, -5.1512e-01,
         -2.7167e-01,  6.8755e-01,  8.5922e-02, -1.4705e-02,  7.4060e-01,
          1.2574e-01, -8.7184e-02, -5.5830e-01,  2.7998e-01, -2.9960e-02,
          5.2273e-01, -6.0473e-01,  1.3639e+00,  7.4814e-01,  8.7402e-01,
         -5.2066e-02,  4.9259e-01,  1.3765e-01,  7.1270e-01, -1.1786e-01,
          7.2326e-01, -1.1462e-01,  3.0809e-01,  8.7026e-01,  1.0988e-01,
          9.9586e-01,  7.8650e-01,  6.